In [9]:
%%writefile utils.py
import os
import pickle
import numpy as np
from deepface import DeepFace
import pyttsx3

def generate_database_embeddings():
    database = {}

    print("[INFO] Generating embeddings for database...")
    for person in os.listdir("database"):
        person_path = os.path.join("database", person)
        if os.path.isdir(person_path):
            embeddings = []
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                try:
                    embedding = DeepFace.represent(
                        img_path=image_path,
                        model_name="Facenet",
                        enforce_detection=True
                    )[0]["embedding"]
                    embedding = np.array(embedding) / np.linalg.norm(embedding)
                    embeddings.append(embedding)
                    print(f"[INFO] Saved embedding for: {image_path}")
                except Exception as e:
                    print(f"[WARN] Could not process {image_path}: {e}")
            if embeddings:
                database[person] = embeddings

    with open("embeddings.pkl", "wb") as f:
        pickle.dump(database, f)
    print("[INFO] Database embeddings saved to embeddings.pkl")


def recognize_face(input_img_path, threshold=0.7):
    with open("embeddings.pkl", "rb") as f:
        database = pickle.load(f)

    try:
        input_embedding = DeepFace.represent(
            img_path=input_img_path,
            model_name="Facenet",
            enforce_detection=False
        )[0]["embedding"]
        input_embedding = np.array(input_embedding) / np.linalg.norm(input_embedding)

        print(f"[DEBUG] Input embedding norm: {np.linalg.norm(input_embedding):.4f}")
        print(f"[DEBUG] First 5 values: {input_embedding[:5]}")

        min_dist = float("inf")
        recognized_id = None

        for person_id, embeddings in database.items():
            for idx, emb in enumerate(embeddings):
                dist = np.linalg.norm(input_embedding - emb)
                print(f"[DEBUG] Distance to {person_id} - sample {idx+1}: {dist:.4f}")
                if dist < min_dist:
                    min_dist = dist
                    recognized_id = person_id

        print(f"[RESULT] Closest match: {recognized_id} (distance = {min_dist:.4f})")

        if min_dist < threshold:
            return True, recognized_id, min_dist
        else:
            return False, None, min_dist

    except Exception as e:
        print("[ERROR] Recognition failed:", e)
        return False, None, None

engine= pyttsx3.init()
def speak(text): 
         try:
             engine.say(text)
             engine.runAndWait()
         except Exception as e:
             print(f"[ERROR] speaking failed: {e}")


def log_attendance(student_id):
    with open("attendance_log.txt", "a") as f:
        f.write(f"{student_id}\n")
    print(f"[INFO] Logged attendance for {student_id}")

Overwriting utils.py


In [3]:
pip install pyttsx3

  Obtaining dependency information for pyttsx3 from https://files.pythonhosted.org/packages/94/df/e1584757c736c4fba09a3fb4f22fe625cc3367b06c6ece221e4b8c1e3023/pyttsx3-2.98-py3-none-any.whl.metadata
  Using cached pyttsx3-2.98-py3-none-any.whl.metadata (3.8 kB)
  Obtaining dependency information for pyobjc>=2.4 from https://files.pythonhosted.org/packages/a9/32/ad08b45fc0ad9850054ffe66fb0cb2ff7af3d2007c192dda14cf9a3ea893/pyobjc-11.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyobjc-core==11.1 from https://files.pythonhosted.org/packages/5a/a7/55afc166d89e3fcd87966f48f8bca3305a3a2d7c62100715b9ffa7153a90/pyobjc_core-11.1-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for pyobjc-framework-libdispatch==11.1 from https://files.pythonhosted.org/packages/b0/cd/1010dee9f932a9686c27ce2e45e91d5b6875f5f18d2daafadea70090e111/pyobjc_framework_libdispatch-11.1-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information f